In [102]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import Polygon, LineString, Point
import math
import warnings
warnings.filterwarnings(action='ignore')
#import descartes
import folium
from tqdm.notebook import tqdm

In [47]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


In [70]:
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')

parkingLot_raw = pd.read_csv('광양시_주차장_공간정보_전처리.csv')
campsite_raw = pd.read_csv('광양시_대중집합시설_야영장_전처리.csv')
traf_parkingLot_raw = pd.read_csv("광양시_도시계획(교통시설)_전처리.csv")
CulPys_raw = gpd.read_file('25.광양시_도시계획(공공문화체육시설).geojson')
chargingStation_raw = pd.read_csv('01.광양시_충전기설치현황.csv')
building_raw = pd.read_csv('광양시_건물정보_전처리.csv') 

In [71]:
#CulPys_raw와 chargingStation_raw에 category컬럼 추가
CulPys_raw['category'] = '공공문화체육시설'
chargingStation_raw['category'] = '충전기'

In [72]:
#충전기를 제외한 카테고리에서 추가할 컬럼인 '완속충전기_설치가능수량'을 충전소데이터에 미리 추가
#완속충전기일 경우 1, 급속충전기일 경우 0을 넣음

chargingStation_raw['완속충전기_설치가능여부'] = np.nan
for i in range(len(chargingStation_raw)):
    if chargingStation_raw['급속/완속'][i] == '완속':
        chargingStation_raw['완속충전기_설치가능여부'][i] = 1
    else:
        chargingStation_raw['완속충전기_설치가능여부'][i] = 0
chargingStation_raw

,충전소명,충전소위치,충전기 운영기관,급속/완속,충전기용량,이용대수,충전기타입,요금정보(원/kw),lon,lat,category,완속충전기_설치가능여부
0,LF스퀘어 광양점,전남 광양시 광양읍 순광로 466 1층,환경부(한국자동차환경협회),급속,200KW 동시충전,2,DC콤보,255.70,127.568300,34.963002,충전기,0.0
1,광양만권경제자유구역청,전남 광양시 광양읍 인덕로 1100(광양시 제2청사),환경부(한국자동차환경협회),급속,50KW,1,DC차데모/AC3상/DC콤보,255.70,127.583323,34.980334,충전기,0.0
2,광양송보파인빌5차아파트,전남 광양시 진등길 55-5 (마동 송보파인빌류 아파트) 501동~506동 상가동,케이티,완속,7kw/h,5,AC완속,170.00,127.689013,34.953759,충전기,1.0
3,광양시 모리스모텔,전남 광양시 광영로 118-1 지상주차장,차지비,완속,7kw/h,2,AC완속,250.00,127.716708,34.964032,충전기,1.0
4,광양시 백운쇼핑센타,전남 광양시 폭포사랑길 99 주차장,차지비,완속,7kw/h,2,AC완속,245.00,127.732440,34.940444,충전기,1.0
5,광양시 블루핸즈(광양정비센터),전남 광양시 광양읍 해광로 888 실외주차장,차지비,완속,7kw/h,1,AC완속,245.00,127.582443,34.963334,충전기,1.0
6,광양시 비치모텔,전남 광양시 진월면 백운1로 389 주차장,차지비,완속,7kw/h,2,AC완속,250.00,127.750981,34.962506,충전기,1.0
7,광양시 포스코 ICT 광양사업소,전남 광양시 태인4길 20 주차장,차지비,완속,7kw/h,1,AC완속,250.00,127.760072,34.941198,충전기,1.0
8,광양시 프라자모텔,전남 광양시 광영로 34,차지비,완속,7kw/h,2,AC완속,250.00,127.725127,34.962147,충전기,1.0
9,광양시 광양읍사무소,전남 광양시 광양읍 남등길 6,한국전기차충전서비스,완속,7kw/h,1,AC완속,255.70,127.580612,34.973057,충전기,1.0


In [91]:
raw_table = pd.concat([parkingLot_raw, traf_parkingLot_raw, campsite_raw, CulPys_raw, chargingStation_raw, building_raw], ignore_index=True)
raw_table

,주차장명칭,구분,DGM_AR,주차면수,유료/무료,주소,lon,lat,geometry,category,...,명칭,충전소명,충전소위치,충전기 운영기관,급속/완속,충전기용량,이용대수,충전기타입,요금정보(원/kw),완속충전기_설치가능여부
0,인동숲앞 주차장,노외,4189.00,161.0,유료,인서리 237-1,127.586883,34.970324,POINT (127.5868828 34.97032367),주차장,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,구터미널주차장,노외,2968.00,125.0,유료,중동 1651,127.695428,34.936252,POINT (127.6954279 34.93625189999999),주차장,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,구경찰서 주차장,노외,2836.00,89.0,무료,읍내리 227-1,127.584586,34.975637,POINT (127.5845858 34.97563677),주차장,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,칠성1지구,노외,1980.00,52.0,무료,칠성리 962-1,127.580683,34.970130,POINT (127.5806828 34.97013011),주차장,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,칠성1지구,노외,1970.00,64.0,무료,칠성리 948-1,127.580707,34.972523,POINT (127.5807067 34.97252282),주차장,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,NaN,NaN,79.95,0.0,NaN,NaN,127.768449,34.985126,POINT (127.7684494316839 34.98512629448562),휴게소,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1210,NaN,NaN,48.00,0.0,NaN,NaN,127.768300,34.984707,POINT (127.7682996761106 34.98470652728316),휴게소,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1211,NaN,NaN,77.49,0.0,NaN,NaN,127.769068,34.985767,POINT (127.769068141161 34.98576670883541),휴게소,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1212,NaN,NaN,54.00,0.0,NaN,NaN,127.769183,34.985841,POINT (127.7691831119503 34.98584074102382),휴게소,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 완속충전기 설치가능 수량(여부) 계산

환경부 2020년 전기자동차 보급 및 충전인프라 구축사업 충전인프라 설치운영지침에 따르면   
해당 장소에 기 설치된 충전기가 없는 경우
#### 해당 장소의 주차단위구획 수× (0.01 + 해당 시‧도의 최근 3년간 신규 차량등록대수 분의 신규 전기차 등록대수) 까지 설치가 가능하다

전라남도청이 공개한 2019년 말 기준 광양시의 지난 3년간의 자동차 등록 현황 대수는    
87,990대(2019년 말) - 78,235대(2016년 9월 기준) = 9755대이다.

In [58]:
#각 행정구역 연도별 등록된 자동차의 수
ElecCar_raw = pd.read_csv('06.광양시_전기차보급현황(연도별,읍면동별).csv')
adm = ElecCar_raw.행정구역.unique()

x = []
y = []
z = []
q = []
for i in range(len(ElecCar_raw)):
    if ElecCar_raw.기준년도[i] == 2017:
        x.append(ElecCar_raw.보급현황[i])
    elif ElecCar_raw.기준년도[i] == 2018:
        y.append(ElecCar_raw.보급현황[i])
    elif ElecCar_raw.기준년도[i] == 2019:
        z.append(ElecCar_raw.보급현황[i])
    else:
        q.append(ElecCar_raw.보급현황[i])


data = {'administration': adm, '2017' : x, '2018' : y , "2019" : z, '2020': q}
ElecCar = pd.DataFrame(data)
ElecCar['2017'].astype('int')
ElecCar['2018'].astype('int')
ElecCar['2019'].astype('int')
ElecCar['2020'].astype('int')
ElecCar      

,administration,2017,2018,2019,2020
0,광양읍,17,1,44,40
1,봉강면,0,0,4,0
2,옥룡면,3,0,3,5
3,옥곡면,3,0,2,3
4,진상면,1,0,2,2
5,진월면,1,0,2,2
6,다압면,0,0,0,0
7,골약동,2,1,1,0
8,중마동,17,8,27,26
9,광영동,3,0,6,3


In [59]:
#각 행정구역별 3년간 신규 전기차 등록대수
elist = []
for i in range(len(ElecCar)):
    a = 0
    for j in range(1, len(ElecCar.columns)-1):
        if ElecCar.loc[i][j] < ElecCar.loc[i][j+1]:
            x = ElecCar.loc[i][j+1] - ElecCar.loc[i][j]
            a += x
    elist.append(a)

administrative = gyboundary.ADM_DR_NM.tolist()
administrative.remove('광양읍')
administrative.insert(0, '광양읍')
data = {'administration': administrative, 'supply' : elist}
ElecCar_Supply = pd.DataFrame(data)
total = sum(ElecCar_Supply.supply)
ElecCar_Supply

,administration,supply
0,광양읍,43
1,봉강면,4
2,옥룡면,5
3,옥곡면,3
4,진상면,2
5,진월면,2
6,다압면,0
7,골약동,0
8,중마동,19
9,광영동,6


In [60]:
#광양시 3년간 신규 전기차 등록대수
total_new_eleccar = ElecCar_Supply['supply'].sum()
total_new_eleccar

93

# 인구분포 & 등록된 자동차 현황

In [86]:
population = gpd.read_file("08.광양시_격자별인구현황(100X100).geojson")
car = gpd.read_file("03.광양시_자동차등록현황_격자(100X100).geojson")

In [88]:
population.loc[population['val'].apply(pd.isna), 'val'] = 0    #NaN값 0으로 치환
population=population[population['val']!=0]                    #인구수가 0인 곳 제외
population_lon_lat = np.stack([population['geometry'].centroid.x, population['geometry'].centroid.y], axis=1).reshape([1, -1, 2])

car = car[car['totale']!=0]             #자동차수가 0인 곳 제외
car_lon_lat = np.stack([car['geometry'].centroid.x, car['geometry'].centroid.y], axis=1).reshape([1, -1, 2])
#차 속도 = 60km/h & 15분 거리 이내의 충전소에 간다고 가정 --> 15분에 15km (15000m)
speed = 60
distance = 60*1000/4

scale = sum([
    *(population['geometry'].bounds['maxx'] - population['geometry'].bounds['minx']),
    *(population['geometry'].bounds['maxy'] - population['geometry'].bounds['miny']),
]) / (2 * len(population)) / 100

# 테이블 함수

In [89]:
def make_table(table):
    table.drop(['주차장명칭', '구분', '유료/무료', '주소', 'PRESENT_SN', 'DGM_NM', 'DGM_LT','명칭','충전소명', '충전소위치', '충전기 운영기관', '충전기용량', '이용대수', '충전기타입','요금정보(원/kw)'], axis=1, inplace=True)
    
    for i in range(len(table)):
        if math.isnan(table['lon'][i]):
            table['lon'][i] = table['geometry'][i].centroid.x
            table['lat'][i] = table['geometry'][i].centroid.y
    table['geometry'] = table.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
    
    table.rename(columns={"DGM_AR":"area"}, inplace=True)         
        
    #계산한 값의 의미는 설치 가능한 완속충전기의 수량이므로 그 값이 1이상이면 설치 가능하다고 생각하여 1로, 1미만이면 0으로 계산.    
    for i in range(len(table)):
        if math.isnan(table['완속충전기_설치가능여부'][i]):
            if table['주차면수'][i] * (0.01 + total_new_eleccar/9755) >= 1:
                table['완속충전기_설치가능여부'][i] = 1
            else:
                table['완속충전기_설치가능여부'][i] = 0
    
    
    lon_lat = np.stack([table.lon,table.lat], axis=1).reshape([-1,1,2])
    
    #인구분포계산
    인구분포_list = []
    population_condition = np.sum((lon_lat - population_lon_lat)**2, axis=2)**0.5 < distance*scale
    인구분포_list = np.array([population.loc[x,'val'].mean() for x in population_condition])
    table['인구분포'] = 인구분포_list    
    
    #등록되어있는 자동차분포 계산
    자동차분포_list = []
    car_condition = np.sum((lon_lat - car_lon_lat)**2, axis=2)**0.5 < distance*scale
    자동차분포_list = np.array([car.loc[x,'totale'].mean() for x in car_condition])
    table['자동차등록현황'] = 자동차분포_list
       
    #소유주체구분 --> 0 : 개인, 1 : 개인아님, 2 : None&법인
    table['소유주체'] = table['소유주체'].fillna(2)
    table['소유주체'] = table['소유주체'].replace({"민간":0,"개인":0, "종종":0, "종교단체":0,"공공":1, "군유지":1,"국유지":1,"시":1,"도유지":1,"법인":2})
    table['소유주체'] = table['소유주체'].astype(int)
    
    table['administration']=np.nan
    for i in range((len(table))):
        for j in range(len(gyboundary)):
            if table.geometry[i].within(gyboundary.geometry[j]) ==True:
                table['administration'][i] = gyboundary.ADM_DR_NM[j]
                
    table['전기차증가량'] = np.nan
    for i in range(len(table)):
        for j in range(len(ElecCar_Supply)):
            if table['administration'][i] == ElecCar_Supply['administration'][j]:
                table['전기차증가량'][i] = ElecCar_Supply['supply'][j]
    
    not_in_Gwangyang = table[table['administration'].isnull()]
    table.drop(table.index[not_in_Gwangyang.index.tolist()], inplace=True)
    
    
    table.reset_index(drop=True,inplace=True)
    return table


In [92]:
table=make_table(raw_table)

In [93]:
table

,area,주차면수,lon,lat,geometry,category,소유주체,급속/완속,완속충전기_설치가능여부,인구분포,자동차등록현황,administration,전기차증가량
0,4189.00,161.0,127.586883,34.970324,POINT (127.5868828 34.97032367),주차장,1,NaN,1.0,82.195652,6.775974,광양읍,43.0
1,2968.00,125.0,127.695428,34.936252,POINT (127.6954279 34.93625189999999),주차장,1,NaN,1.0,75.368805,6.342394,중마동,19.0
2,2836.00,89.0,127.584586,34.975637,POINT (127.5845858 34.97563677),주차장,1,NaN,1.0,81.790323,6.704258,광양읍,43.0
3,1980.00,52.0,127.580683,34.970130,POINT (127.5806828 34.97013011),주차장,1,NaN,1.0,83.303086,6.753846,광양읍,43.0
4,1970.00,64.0,127.580707,34.972523,POINT (127.5807067 34.97252282),주차장,1,NaN,1.0,83.083692,6.749640,광양읍,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,79.95,0.0,127.768449,34.985126,POINT (127.7684494 34.98512629),휴게소,2,NaN,0.0,81.355035,7.031979,진월면,2.0
1206,48.00,0.0,127.768300,34.984707,POINT (127.7682997 34.98470653),휴게소,2,NaN,0.0,81.355035,7.029294,진월면,2.0
1207,77.49,0.0,127.769068,34.985767,POINT (127.7690681 34.98576671),휴게소,2,NaN,0.0,81.419635,7.031979,진월면,2.0
1208,54.00,0.0,127.769183,34.985841,POINT (127.7691831 34.98584074),휴게소,1,NaN,0.0,81.419635,7.031979,진월면,2.0


In [95]:
table.to_csv("표준화안함.csv", index=False, encoding='cp949')

## 표준화 함수

In [111]:
population_mean = table['인구분포'].mean()
population_std = table['인구분포'].std()
car_mean = table['자동차등록현황'].mean()
car_std = table['자동차등록현황'].std()
eleccar_increment_mean = table['전기차증가량'].mean()
eleccar_increment_std = table['전기차증가량'].std()

def preprocess(df):
    df = df.copy()
    df['인구분포'] = (df['인구분포'] - population_mean) / (population_std + 1e-3)
    df['자동차등록현황'] = (df['자동차등록현황'] - car_mean) / (car_std + 1e-3)    
    df['전기차증가량'] = (df['전기차증가량'] - eleccar_increment_mean) / (eleccar_increment_std + 1e-3)    
    return df

In [112]:
table = preprocess(table)
table

,area,주차면수,lon,lat,geometry,category,소유주체,급속/완속,완속충전기_설치가능여부,인구분포,자동차등록현황,administration,전기차증가량
0,4189.00,161.0,127.586883,34.970324,POINT (127.58688 34.97032),주차장,1,NaN,1.0,0.652892,0.607904,광양읍,1.262407
1,2968.00,125.0,127.695428,34.936252,POINT (127.69543 34.93625),주차장,1,NaN,1.0,-0.278347,-0.157118,중마동,-0.162234
2,2836.00,89.0,127.584586,34.975637,POINT (127.58459 34.97564),주차장,1,NaN,1.0,0.597602,0.481367,광양읍,1.262407
3,1980.00,52.0,127.580683,34.970130,POINT (127.58068 34.97013),주차장,1,NaN,1.0,0.803955,0.568861,광양읍,1.262407
4,1970.00,64.0,127.580707,34.972523,POINT (127.58071 34.97252),주차장,1,NaN,1.0,0.774028,0.561439,광양읍,1.262407
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,79.95,0.0,127.768449,34.985126,POINT (127.76845 34.98513),휴게소,2,NaN,0.0,0.538225,1.059607,진월면,-1.171355
1206,48.00,0.0,127.768300,34.984707,POINT (127.76830 34.98471),휴게소,2,NaN,0.0,0.538225,1.054870,진월면,-1.171355
1207,77.49,0.0,127.769068,34.985767,POINT (127.76907 34.98577),휴게소,2,NaN,0.0,0.547037,1.059607,진월면,-1.171355
1208,54.00,0.0,127.769183,34.985841,POINT (127.76918 34.98584),휴게소,1,NaN,0.0,0.547037,1.059607,진월면,-1.171355


In [80]:
table.to_csv("테이블.csv", index=True, encoding='cp949')

In [113]:
table_gpd = GeoDataFrame(table, crs="EPSG:4326", geometry=[shapely.geometry.Point(xy) for xy in zip(table.lon, table.lat)])

In [114]:
table_gpd.to_file("table.geojson", driver='GeoJSON')